In [78]:
#s1="'12.12psds"
### take the first number of the string
def mstr(ms):
    sal=""
    #ms=ms[1:]
    for i in range(len(ms)):
        if (ms[i] in '0123456789.'):
            sal=sal+ms[i]
           # print (ms[i])
        else: break
    return sal
#mstr(s1)

#s1="'12.12psds"
### take the first number of the string
def mstr2(ms):
    sal=""
    #ms=ms[1:]
    for i in range(len(ms)):
        if (ms[i] != '\\' and ms[i] != ',' ):
            sal=sal+ms[i]
           # print (ms[i])
        else: break
    return sal
#mstr(s1)

In [81]:
#!/usr/bin/env python
# coding: utf-8

# In[107]:


import os
import openai
import matplotlib.pyplot as plt
import numpy as np
openai.api_key = "sk-LopWyMKqQ2aCySPMeFSZLyrAur0a8EACzgbHBXMK"
from sklearn.datasets import fetch_openml
from random import randrange,sample

from sklearn import tree

#### ZEROSHOT
zeroshot=True

adult = fetch_openml(data_id=43436) #61 iris, 196 MPG 43436 adult
results=[]

# In[108]:


from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
mdatos=adult['data']
#remove missing
mdatos=mdatos.dropna()
#remove classes with few elements


#cl="race"
nc=-1
for cl in mdatos.columns:
  
  nc=nc+1
  print("column:" +cl)
  
  classif=False
  if (mdatos[cl].dtype=="O"):
      classif=True
      datos=mdatos.groupby(cl).filter(lambda x: len(x) > 10)
  else:
      classif=False
      datos=mdatos

  datos = datos.sample(frac=1).reset_index(drop=True)
  X=datos.loc[:, datos.columns != cl] # Features
  y=datos[cl]
   
  
  
  # In[109]:
  
  
  from sklearn.pipeline import Pipeline
  from sklearn.preprocessing import OneHotEncoder
  from sklearn.impute import SimpleImputer
  from sklearn.compose import ColumnTransformer
  cols = datos.columns
  
  num_cols = X._get_numeric_data().columns
  categorical_columns  = [col for col in X.columns if X[col].dtype=="O"]
  
  
  numerical_columns = list(num_cols)
  
  #X = X[categorical_columns + numerical_columns]
  
  
  
  #categorical_columns = X._get_numeric_data().columns
  
  
  #X = X[categorical_columns + numerical_columns]
  
  #X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
  
  

  
  #fig, ax = plt.subplots()
  #ax.boxplot(
  #    result.importances[sorted_idx].T, vert=False, labels=X_test.columns[sorted_idx]
  #)
  #ax.set_title("Permutation Importances (test set)")
  #fig.tight_layout()
  #plt.show()

  
  
  
  num_cols = X._get_numeric_data().columns
  categorical_columns  = [col for col in X.columns if X[col].dtype=="O"]
  
  
  numerical_columns = list(num_cols)
  
  #X = X[categorical_columns + numerical_columns]
  
  
  
  categorical_encoder = OneHotEncoder(handle_unknown="ignore")
  numerical_pipe = Pipeline([("imputer", SimpleImputer(strategy="mean"))])
  
  preprocessing = ColumnTransformer(
      [
          ("cat", categorical_encoder, categorical_columns),
          ("num", numerical_pipe, numerical_columns),
      ]
   )
  
  if (classif):
   dt = Pipeline(
      [
          ("preprocess", preprocessing),
          ("classifier", tree.DecisionTreeClassifier(random_state=42)),
      ]
   )
  else:
   dt = Pipeline(
      [
          ("preprocess", preprocessing),
          ("classifier", tree.DecisionTreeRegressor(random_state=42)),
      ]
   ) 
  
  
  
  
  # In[115]:
  
  
  n=10 #repetitions
  tam=11 #prompt train + 1 test
  
  responses=[]
  correct=[]
  responsesg=[]
  
  for ij in range(n):
    #  print (ij)
     
   #   if (classif):
   #    X1_train, X1_test, y1_train, y1_test = train_test_split(X, y,train_size=tam, stratify=y, random_state=ij)
   #   else:
   #    X1_train, X1_test, y1_train, y1_test = train_test_split(X, y,train_size=tam, random_state=ij)
    
      vals=sample(range(0, len(X)), tam)
      X1_train=X.loc[vals[1:], :]
      X1_test=y.loc[vals[1:]]
      y1_train=X.loc[vals[0],:]
      y1_test=y.iloc[vals[0]]
      ### DT
    ##  dt.fit(X1_train, y1_train) No va!
     ## predictions = dt.predict(X1_test)
     ## predictions[0]==y1_test.iloc[0]
      #print(predictions[0])
      #print(y1_test.iloc[0])
     # correct.append(y1_test.iloc[0]) 
     # responses.append(predictions[0])
      ### GPT3
      prompt=""
      
              #Build Prompt
      itst=sample(range(0, len(datos)), tam)
      lc=[item for item in range(0, len(mdatos.columns))]
      lc.remove(nc)
      #traim
      for i in range (0,(tam-1)):
         itr=itst[i]
            #train
         for j in range (0,len(lc)):
                      prompt=prompt+cols[lc[j]]+":"+str(datos.iloc[itr,lc[j]])+", "
         prompt=prompt+cols[nc]+":"+str(datos.iloc[itr,nc])+", "   
         prompt=prompt[:-2]+' \n '

            
      if (zeroshot): prompt="" #### ZERO SHOT!     
        #test
      itr=itst[tam-1]
      for j in range (0,len(lc)):
                      prompt=prompt+cols[lc[j]]+":"+str(datos.iloc[itr,lc[j]])+", "
      prompt=prompt+cols[nc]+":"
      correct.append(datos.iloc[itr,nc]) 

      response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        temperature=0,
        max_tokens=64,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
          )

      #Response
     # if (response["choices"][0]["text"].split('\n')[0]==y1_test.iloc[0]):
     #     accg=accg+1
      #responsesg.append(response["choices"][0]["text"].split('\n')[0])   
      if (zeroshot and classif==False):
              mres=mstr(response["choices"][0]["text"].split('\n')[0])   
      else:
            mres=mstr2(response["choices"][0]["text"].split('\n')[0])
      responses.append(mres)   
      responsesg.append(mres)   
  
  # In[116]:
  
  
  acc=0
  accg=0
  if classif:
   for i in range(n):  
          if (responses[i].strip()==correct[i].strip()):
              acc=acc+1
          if (responsesg[i].strip()==correct[i].strip()):
              accg=accg+1
  
  if not(classif):
   for i in range(n):
          acc=acc+abs(float(responses[i])-float(correct[i]))
          accg=accg+abs(float(responsesg[i])-float(correct[i]))
               
  
  
  # In[117]:
  
  if classif:  
   print("accuracy gpt3")
   print(float(accg)/n)
   print("accuracy DT")
   print(float(acc)/n)
   results.append((cl,n,float(accg)/n,float(acc)/n))

  if not(classif):
    print("MAE gpt3")
    print(float(accg)/n)
    print("MAE DT")
    print(float(acc)/n)
    results.append((cl,n,float(accg)/n,float(acc)/n))

  
  
  
  


column:age
MAE gpt3
11.1
MAE DT
11.1
column:workclass
accuracy gpt3
0.7
accuracy DT
0.7
column:fnlwgt
MAE gpt3
68407.1
MAE DT
68407.1
column:education
accuracy gpt3
0.7
accuracy DT
0.7
column:education.num
MAE gpt3
0.0
MAE DT
0.0
column:marital.status
accuracy gpt3
0.3
accuracy DT
0.3
column:occupation
accuracy gpt3
0.2
accuracy DT
0.2
column:relationship
accuracy gpt3
0.7
accuracy DT
0.7
column:race
accuracy gpt3
0.9
accuracy DT
0.9
column:sex
accuracy gpt3
0.9
accuracy DT
0.9
column:capital.gain
MAE gpt3
861.4
MAE DT
861.4
column:capital.loss
MAE gpt3
156.4
MAE DT
156.4
column:hours.per.week
MAE gpt3
13.5
MAE DT
13.5
column:native.country
accuracy gpt3
0.5
accuracy DT
0.5
column:income
accuracy gpt3
0.0
accuracy DT
0.0


In [79]:
mstr2(response["choices"][0]["text"].split('\n')[0])

'Private'

In [83]:
correct

['=50K', '50K', '=50K', '50K', '=50K', '50K', '50K', '50K', '=50K', '=50K']

In [82]:
responses

['<=50K',
 '>50K',
 '<=50K',
 '<=50K',
 '<=50K',
 '<=50K',
 '<=50K',
 '<=50K',
 '<=50K',
 '<=50K']

In [59]:
response

<OpenAIObject text_completion id=cmpl-5Ibpz7A4wQHyxxgurGGR0vwY4P0v3 at 0x7fdadc98b180> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "<=50K\\n', '\\n']\n\n\n\n> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating"
    }
  ],
  "created": 1655117035,
  "id": "cmpl-5Ibpz7A4wQHyxxgurGGR0vwY4P0v3",
  "model": "text-davinci-002",
  "object": "text_completion"
}

In [175]:
prompt

'age:45.0, fnlwgt:242391.0, education:Bachelors, education.num:13.0, marital.status:Married-civ-spouse, occupation:Exec-managerial, relationship:Husband, race:White, sex:Male, capital.gain:0.0, capital.loss:0.0, hours.per.week:40.0, native.country:United-States, income:50K, workclass:Private \n age:27.0, fnlwgt:29261.0, education:HS-grad, education.num:9.0, marital.status:Married-AF-spouse, occupation:Craft-repair, relationship:Husband, race:White, sex:Male, capital.gain:0.0, capital.loss:0.0, hours.per.week:40.0, native.country:United-States, income:50K, workclass:Private \n age:20.0, fnlwgt:482732.0, education:HS-grad, education.num:9.0, marital.status:Never-married, occupation:Adm-clerical, relationship:Other-relative, race:White, sex:Male, capital.gain:0.0, capital.loss:0.0, hours.per.week:24.0, native.country:United-States, income:=50K, workclass:Private \n age:49.0, fnlwgt:189123.0, education:HS-grad, education.num:9.0, marital.status:Married-civ-spouse, occupation:Craft-repair, 

In [71]:
Xn.loc[[11548, 4707, 7955], :]


age         workclass    fnlwgt     education  education.num  \
11548  55.0           Private  323706.0  Some-college           10.0   
4707   33.0           Private  112358.0  Some-college           10.0   
7955   47.0  Self-emp-not-inc   88500.0       HS-grad            9.0   

           marital.status       occupation   relationship   race   sex  \
11548  Married-civ-spouse  Exec-managerial        Husband  White  Male   
4707        Never-married    Other-service  Not-in-family  White  Male   
7955   Married-civ-spouse  Farming-fishing        Husband  White  Male   

       capital.gain  capital.loss  hours.per.week native.country  
11548           0.0        1902.0            40.0  United-States  
4707            0.0           0.0            32.0  United-States  
7955            0.0           0.0            35.0  United-States

In [114]:
itst=sample(range(0, len(X)), tam)

In [115]:
itst


[22233, 12171, 1120, 15846, 6167, 26495, 26450, 4995, 17069, 22166]

In [102]:

y1_test

42.0

In [86]:
vals[0]

24417